In [8]:
%%capture
!pip3 install pandas
!pip3 install numpy
!pip3 install matplotlib

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#contains movie id and corresponding imdbId and tmdbId
links = pd.read_csv('links.csv')

# contains movieId title and genres that the movie falls under
movies = pd.read_csv('movies.csv')

# contains ratings for movieId by users with a timestampe
ratings = pd.read_csv('ratings.csv')

#contains UserId associated with a movieId with the tag the user rated with and a time stamp
tags = pd.read_csv('tags.csv')

In [3]:
#drops userId and timestamp not needed
ratings.drop(['userId','timestamp'], axis = 1, inplace = True)
#adds a column for mean and count for the rating 
ratings = ratings.groupby('movieId').agg(['mean','count'])
#renames columns
ratings.columns = ['Average_Rating', 'Total_Rating']

# merges ratings with links csv by movieId
movies = movies.merge(ratings, how = 'inner', on = 'movieId')
movies = movies.merge(links,  how ='inner', on = 'movieId')

In [4]:
#creates a dictionary for title and year 
moviesByYear = []
for x in range(len(movies)):
    moviesByYear.append([
        int(movies.movieId[x]), 
        str(movies.title[x][0:movies.title[x].rfind('(')]).rstrip(),
        int(movies.title[x][movies.title[x].rfind('(')+1:movies.title[x].rfind(')')]).rstrip()
    ])

AttributeError: 'int' object has no attribute 'rstrip'

In [5]:
#replaces current title with new title and year
moviesByYearDf = pd.DataFrame(moviesByYear, columns=['movieId',"title","year"])
movies.drop(columns='title', inplace = True)
movies = moviesByYearDf.merge(movies, how = 'inner', on = 'movieId')
movies = movies.round({'Average_Rating': 4})

In [6]:
pd.options.mode.chained_assignment = None
for x in range(len(movies)):
    if movies.genres[x].find('|') != -1:
        movies.genres[x] = movies.genres[x][:movies.genres[x].find('|')]

In [7]:
movies.to_csv('DataforMovies.csv')

Unsure how to implement the other 2 files as of now, use pandas stack maybe ?

In [ ]:
#NOTE STILL FIGURING OUT HOW TO USE THIS PART
#Maybe get top 3 relevant genres to movie and merge those to movies

#Contains movieid tag id and relevance
genome_scores = pd.read_csv('genome-scores.csv')

#contains tagId and tag looks like genres
genome_tags = pd.read_csv('genome-tags.csv')

In [ ]:
#Adds a column for genre which is based of genome_tags.tag 
genome_scores['genre'] = genome_scores['tagId'].map(genome_tags.set_index('tagId')['tag'])
#Sorts by ascending
genome_scores.sort_values(by="relevance", ascending = False, inplace = True)
genome_scores.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True)
genome_scores.reset_index(inplace = True)
genome_scores.drop(columns=['index'], inplace =True)

In [ ]:

print(genome_scores.head())

In [ ]:
movies['genres'].str.contains('|', regex = False)